In [1]:
from random import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from ipywidgets import interact_manual
from matplotlib import interactive
import bisect
%matplotlib notebook

In [2]:
from kivy.app import App
from kivy.uix.widget import Widget
from kivy.core.window import Window
from kivy.uix.button import Button
from kivy.graphics import Color, Ellipse, Line, Rectangle

Window.clearcolor = (1, 1, 1, 1)

class MyPaintWidget(Widget):

    def on_touch_down(self, touch):
        color = (1, 1, 0)
        with self.canvas:
            Color(*color, mode='hsv')
            d = 5
            Ellipse(pos=(touch.x - d / 2, touch.y - d / 2), size=(d, d))
            touch.ud['line'] = Line(points=(touch.x, touch.y))

    def on_touch_move(self, touch):
        touch.ud['line'].points += [touch.x, touch.y]

        
class MyPaintApp(App):

    def build(self):
        parent = Widget()
        self.painter=MyPaintWidget(size=[i/1.2 for i in Window.size])

        clearbtn = Button(text='Clear', pos=(700, 0))
        savebtn = Button(text='Save', pos=(700, parent.height))

        with self.painter.canvas:
            Color(1, 0, 0, 0.3)
            Rectangle(pos=self.painter.pos, size=self.painter.size)
        
        clearbtn.bind(on_release=self.clear_canvas)
        savebtn.bind(on_release=self.save_canvas)
        
        parent.add_widget(self.painter)
        parent.add_widget(clearbtn)
        parent.add_widget(savebtn)
    
        return parent
        

    def save_canvas(self, obj):
        self.painter.export_to_png('a.png')    
    
    def clear_canvas(self, obj):
        self.painter.canvas.clear()


if __name__ == '__main__':
    MyPaintApp().run()

[INFO   ] [Logger      ] Record log in /home/infinity/.kivy/logs/kivy_18-07-15_0.txt
[INFO   ] [Kivy        ] v1.10.0
[INFO   ] [Python      ] v3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]
[INFO   ] [Factory     ] 194 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_pygame, img_pil, img_gif (img_ffpyplayer ignored)
[INFO   ] [Window      ] Provider: pygame(['window_egl_rpi'] ignored)
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] Backend used <gl>
[INFO   ] [GL          ] OpenGL version <b'3.0 Mesa 17.2.8'>
[INFO   ] [GL          ] OpenGL vendor <b'X.Org'>
[INFO   ] [GL          ] OpenGL renderer <b'AMD MULLINS (DRM 2.50.0 / 4.13.0-45-generic, LLVM 5.0.0)'>
[INFO   ] [GL          ] OpenGL parsed version: 3, 0
[INFO   ] [GL          ] Shading version <b'1.30'>
[INFO   ] [GL          ] Texture max size <16384>
[INFO   ] [GL          ] Texture max units <32>
[INFO   ] [Window      ] virtual keyboard not allowed,

In [3]:
%matplotlib notebook
im = Image.open('a.png')
pix = im.load()

X_old = []
Y_old = []
default = pix[0,0]
for x in range(im.size[0]):
    for y in range(im.size[1]):
        if pix[x,y]!=default:
            X_old.append(x)
            Y_old.append(im.size[1] - y)
            
X_old = np.array(X_old)
Y_old = np.array(Y_old)

prev_x = X_old[0]
prev_y = Y_old[0]
X = []
Y = []

len_run = 1
y_sum = prev_y
for curr_x,curr_y in zip(X_old[1:],Y_old[1:]):
    if curr_x == prev_x:
        len_run+=1
        y_sum+=curr_y
    else:
        X.append(prev_x)
        Y.append(y_sum/len_run)
        y_sum = curr_y
        len_run = 1
    prev_x = curr_x
    
X.append(prev_x)
Y.append(y_sum/len_run)

X = np.array(X)
Y = np.array(Y)

In [4]:
def nor_val(x,mean,variance):
    return np.exp((-(x-mean)**2)/(2*variance))/(2*np.pi*variance)**0.5

def cover(y_nor,y):
    for i,j in zip(y_nor,y):
        if i < j:
            return False
    return True

def find_index_(sample,X):
    ind = bisect.bisect(X,sample)
    val = X[ind]
    if val == sample:
        return ind
    else:
        return ind-1

In [5]:
@interact_manual(lower_x_limit = '-10', upper_x_limit = '10', Num_samples = '1000', PLOT = True, ret = False, pr = True)
def normalize(lower_x_limit = '-10',upper_x_limit = '10',Num_samples ='1000' , PLOT = True, ret = False,pr = True):
    Num_samples = int(Num_samples)
    a = float(lower_x_limit)
    b = float(upper_x_limit)
    size_x = X.shape[0]
    X_shrinked = np.linspace(a,b,size_x)

    delta = X_shrinked[1] - X_shrinked[0]
    area = (np.sum(Y) - 0.5 * (Y[0] + Y[-1])) * delta
    Y_shrinked = Y/area
    f_max = Y_shrinked.max()
    
    if PLOT == True:
        plt.close(1)
        plt.close(2)
        plt.close(3)
        plt.close(4)
        f = plt.figure(1)
        plt.plot(X_shrinked, Y_shrinked,c = 'r',linewidth = 2)
        delt = (b - a)*0.1
        low = a - delt
        high = b + delt
        x_width = high - low
        high_y = f_max * 1.5
        plt.xlim(low,high)
        plt.ylim(0,high_y)
        plt.axvline(x=a,linewidth = 2, c='r',ymax=Y_shrinked[0]/high_y)
        plt.axvline(x=b, linewidth = 2, c='r',ymax=Y_shrinked[-1]/high_y)
        plt.title("pdf")
        interactive(True)
        f.show()
    
    
    

    #Trying out Uniform Distribution

    highest_val_uniform = 1/(b-a)
    M_uni = f_max/highest_val_uniform
    Uni_acc = 1/M_uni
    
    if PLOT == True:
        g = plt.figure(2)
        plt.plot(X_shrinked, Y_shrinked,c = 'r',linewidth = 2,label='Original pdf')
        delt = (b - a)*0.1
        low = a - delt
        high = b + delt
        x_width = high - low
        high_y = f_max * 1.5
        plt.xlim(low,high)
        plt.ylim(0,high_y)
        plt.axvline(x=a,linewidth = 2, c='r',ymax=Y_shrinked[0]/high_y)
        plt.axvline(x=b, linewidth = 2, c='r',ymax=Y_shrinked[-1]/high_y)
        plt.title("Considering Uniform Cover")
        plt.axhline(y=f_max,linewidth= 2, xmin = delt/x_width, xmax = 1 - delt/x_width, label = 'Uniform Cover')
        plt.axvline(x=a,linewidth = 2,ymax=f_max/high_y)
        plt.axvline(x=b, linewidth = 2,ymax=f_max/high_y)
        print("Acceptance Ratio = ",1/M_uni)
        plt.legend()
        interactive(True)
        g.show()
        
    #Trying out Normal distribution
    
    Delta = (Y_shrinked[1:] - Y_shrinked[:-1])/(X_shrinked[1] - X_shrinked[0])
    Delta_zero = Delta == 0
    X_criticals = X_shrinked[1:][Delta_zero]
    Y_criticals = Y_shrinked[1:][Delta_zero]
    
    prev_x = X_criticals[0]
    prev_y = Y_criticals[0]
    X_c = []
    Y_c = []
    
    len_run = 1
    x_sum = prev_x
    for curr_x,curr_y in zip(X_criticals[1:],Y_criticals[1:]):
        if curr_y == prev_y:
            len_run+=1
            x_sum+=curr_x
        else:
            Y_c.append(prev_y)
            X_c.append(x_sum/len_run)
            x_sum = curr_x
            len_run = 1
        prev_y = curr_y

    Y_c.append(prev_y)
    X_c.append(x_sum/len_run)

    X_c = np.array(X_c)
    Y_c = np.array(Y_c)

    Ind = Y_c.argsort()[-20:][::-1]
    u=0
    den=0
    for i in Ind:
        u+=X_c[i] *Y_c[i]
        den+=Y_c[i]
        
    u/=den
    
    for M in np.linspace(1,10,91):
        for variance in np.linspace(1,(b-a)**2,100):
            y_nor = nor_val(X_shrinked,u,variance)*M
            res = cover(y_nor,Y_shrinked)    #check whether y_nor covers Y_shrinked
            if res == True:
                break
        if res == True:
            break
            
    if res == False:
        if pr == True:
            print("Normal cover not feasible")
    
    Norm_acc = 1/M
    
    if PLOT == True:
        h = plt.figure(3)
        plt.plot(X_shrinked, Y_shrinked,c = 'r',linewidth = 2,label='Original pdf')
        delt = (b - a)*0.1
        low = a - delt
        high = b + delt
        x_width = high - low
        high_y = f_max * 5
        plt.xlim(low,high)
        plt.ylim(0,high_y)
        plt.axvline(x=a,linewidth = 2, c='r',ymax=Y_shrinked[0]/high_y)
        plt.axvline(x=b, linewidth = 2, c='r',ymax=Y_shrinked[-1]/high_y)
        plt.title("Considering Normal Cover")
        
        plt.axvline(x=u,linestyle = '--',c='g',label='Chosen mean of normal cover')
        plt.plot(X_shrinked,y_nor)
        
        print("Acceptance Ratio = ",1/M)
        print("Mean = ",u)
        print("Variance = ",variance)
        print("M = ",M)
        plt.legend()
        interactive(True)
        h.show()
     
    
    
    counter = 0
    coun = 0
    Result = []
    
    if Uni_acc >= Norm_acc:
        if pr == True:
            print("\n\nChoosing Uniform cover.")
        val_cover = f_max
        while(counter<Num_samples):
            coun+=1
            sample = np.random.rand()*(b-a)+a
            
            index_ = find_index_(sample,X_shrinked)
            previous_y = Y_shrinked[index_]

            if index_<(X_shrinked.shape[0] - 1):
                next_y = Y_shrinked[index_ + 1]
                slope = (next_y - previous_y)/(delta)
                val_pdf = previous_y + slope * (sample - X_shrinked[index_])

            else:
                val_pdf = previous_y
                
            g1 = np.random.rand()*val_cover
                
            if g1<=val_pdf:
                Result.append(sample)
                counter+=1
                
        if PLOT == True:
            q = plt.figure(4)
            plt.hist(Result,bins = int(Num_samples/10))
            plt.title("Histogram plot of the samples generated")
            plt.savefig("Histogram plot of the samples generated")
            interactive(False)
            q.show()        
        
        if pr == True:
            print("Generated ",coun," samples out of which ",counter," samples were accepted")
        
        if ret == True:
            return np.array(Result)
        
        
    else:
        if pr == True:
            print("\n\nChoosing Normal cover.")
        
        Mean = u
        Variance = variance
        
        while(counter<Num_samples):
            coun+=1
            sample = np.random.randn()*(Variance**0.5)+Mean
            if sample>=a and sample<=b:
                index_ = find_index_(sample,X_shrinked)
                previous_y = Y_shrinked[index_]
                
                if index_<(X_shrinked.shape[0] - 1):
                    next_y = Y_shrinked[index_ + 1]
                    slope = (next_y - previous_y)/(delta)
                    val_pdf = previous_y + slope * (sample - X_shrinked[index_])
      
                else:
                    val_pdf = previous_y
                
                val_cover = nor_val(sample,Mean,Variance)
                
                g1 = np.random.rand()*val_cover
                
                if g1<=val_pdf:
                    Result.append(sample)
                    counter+=1
            
        if PLOT == True:
            q = plt.figure(4)
            plt.hist(Result,bins = int(Num_samples/10))
            plt.title("Histogram plot of the samples generated")
            plt.savefig("Histogram plot of the samples generated")
            interactive(False)
            q.show()
            
        if pr == True:
            print("Generated ",coun," samples out of which ",counter," samples were accepted")
        
        if ret == True:
            return np.array(Result)
        

interactive(children=(Text(value='-10', description='lower_x_limit'), Text(value='10', description='upper_x_li…

In [35]:
@interact_manual(N = '100',num_samples = '100')
def median_and_mean_distributions(N = '100',num_samples = '100'):
    
    med = []
    mea = []
    for i in range(int(num_samples)):
        x = normalize(lower_x_limit = '-10',upper_x_limit = '10',Num_samples =N , PLOT = False, ret = True,pr = False)
        med.append(np.median(x))
        mea.append(np.mean(x))
        if (i%5 == 0):
            print(' '*len(str(i)),end='\r')
            print(i,end='\r')
            
        
    med = np.array(med)
    mea = np.array(mea)
    plt.close(5)
    plt.close(6)
    
    N = int(N)
    
    f = plt.figure(5)
    plt.hist(med,bins = N//10,label="Histogram plot")
    Title = "Distribution of median\nExpected value of median is " + str(np.mean(med)) 
    plt.title(Title)
    plt.legend()
    interactive(True)
    f.show()
    
    g = plt.figure(6)
    plt.hist(mea,bins = N//10,label="Histogram plot")
    Title = "Distribution of mean\nExpected value of mean is " + str(np.mean(mea)) 
    plt.title(Title)
    plt.legend()
    interactive(False)
    g.show()

interactive(children=(Text(value='100', description='N'), Text(value='100', description='num_samples'), Button…